# Analyse XPS Data with Python
For analysing X-ray Photoelectron Spectroscopy (XPS) data and performing peak fitting in Python, a good approach is to use libraries like numpy, scipy, matplotlib, and lmfit. Here’s an outline of how you can approach this:

### 1. Load the XPS Data
XPS data usually consists of binding energy (BE) values and intensity counts. If your data is in a CSV or text file, you can use pandas to load it.

In [ ]:
import pandas as pd

# Load the data assuming columns are "Binding Energy" and "Intensity"
xps_data = pd.read_csv('xps_data.csv')
binding_energy = xps_data['Binding Energy']
intensity = xps_data['Intensity']

### 2. Plot the XPS Data
You can use matplotlib to visualize the spectrum.

In [ ]:
import matplotlib.pyplot as plt

plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

### 3. Peak Fitting
The peaks in XPS can be fitted using Gaussian, Lorentzian, or Voigt profiles. A combination of these is typically used to fit overlapping peaks. `scipy.optimize.curve_fit` or `lmfit` can help with non-linear curve fitting. For Gaussian fitting:

In [ ]:
import numpy as np
from scipy.optimize import curve_fit

# Gaussian function definition
def gaussian(x, amp, cen, width):
    return amp * np.exp(-(x - cen)**2 / (2 * width**2))

# Guess initial parameters for fitting
initial_guess = [max(intensity), binding_energy[np.argmax(intensity)], 1.0]

# Perform the fit
popt, pcov = curve_fit(gaussian, binding_energy, intensity, p0=initial_guess)

# Plot the fit
fit_intensity = gaussian(binding_energy, *popt)
plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.plot(binding_energy, fit_intensity, label='Fitted Peak', linestyle='--')
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

For multiple peaks, you can define a sum of Gaussians or Lorentzians and fit them:

In [ ]:
# Define a sum of Gaussians for multiple peak fitting
def multiple_gaussians(x, *params):
    y = np.zeros_like(x)
    for i in range(0, len(params), 3):
        amp, cen, width = params[i:i+3]
        y += gaussian(x, amp, cen, width)
    return y

# Initial guesses for multiple peaks (amp1, cen1, width1, amp2, cen2, width2, ...)
initial_guess = [max(intensity), binding_energy[np.argmax(intensity)], 1.0, 
                 0.5 * max(intensity), binding_energy[np.argmax(intensity)] - 2, 1.0]

# Perform the fit
popt, pcov = curve_fit(multiple_gaussians, binding_energy, intensity, p0=initial_guess)

# Plot the fit
fit_intensity = multiple_gaussians(binding_energy, *popt)
plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.plot(binding_energy, fit_intensity, label='Fitted Peaks', linestyle='--')
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

### 4. Optimization Tools
You can also use `lmfit` for better parameter control and uncertainty estimates.

## Step-by-step `lmfit`

Install the package:

`pip install lmfit`

### 1. Defining the Peak Model
You can use lmfit.Model to define a peak (e.g., Gaussian, Lorentzian, or Voigt). For XPS data, peaks are commonly fitted with Gaussian or Lorentzian functions, or a mix of both (Voigt).

Here’s an example of using a Gaussian model:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from lmfit.models import GaussianModel, LorentzianModel

# Load the XPS data
xps_data = pd.read_csv('xps_data.csv')
binding_energy = xps_data['Binding Energy']
intensity = xps_data['Intensity']

# Create a Gaussian model
gaussian_model = GaussianModel()

# Set initial parameters: amplitude, center, and sigma (width)
params = gaussian_model.make_params(amplitude=max(intensity), center=binding_energy[np.argmax(intensity)], sigma=1.0)

# Perform the fit
result = gaussian_model.fit(intensity, params, x=binding_energy)

# Plot the original data and the fit
plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.plot(binding_energy, result.best_fit, label='Gaussian Fit', linestyle='--')
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

# Print the fitting report
print(result.fit_report())

### 2. Fitting Multiple Peaks (Composite Model)
For complex spectra with multiple peaks, you can combine multiple models (e.g., a combination of Gaussian and Lorentzian functions). Here’s an example of fitting a spectrum with two Gaussian peaks:

In [ ]:
from lmfit import Model
from lmfit.models import GaussianModel, LorentzianModel

# Create a model composed of two Gaussian peaks
gauss1 = GaussianModel(prefix='g1_')
gauss2 = GaussianModel(prefix='g2_')

# Composite model
model = gauss1 + gauss2

# Make parameters for the two Gaussian peaks
params = model.make_params()

# Set initial guesses for the two peaks
params['g1_amplitude'].set(value=max(intensity), min=0)
params['g1_center'].set(value=binding_energy[np.argmax(intensity)], min=min(binding_energy), max=max(binding_energy))
params['g1_sigma'].set(value=1, min=0)

params['g2_amplitude'].set(value=0.5 * max(intensity), min=0)
params['g2_center'].set(value=binding_energy[np.argmax(intensity)] - 2, min=min(binding_energy), max=max(binding_energy))
params['g2_sigma'].set(value=1, min=0)

# Perform the fit
result = model.fit(intensity, params, x=binding_energy)

# Plot the data and the fits
plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.plot(binding_energy, result.best_fit, label='Composite Fit', linestyle='--')

# Plot each individual component
plt.plot(binding_energy, result.eval_components()['g1_'], label='Peak 1', linestyle=':')
plt.plot(binding_energy, result.eval_components()['g2_'], label='Peak 2', linestyle=':')

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

# Print the fitting report
print(result.fit_report())

### 3. Fitting Voigt Peaks (Lorentzian + Gaussian)
If you need to fit Voigt peaks (which combine Gaussian and Lorentzian), you can modify the model like this:

In [ ]:
from lmfit.models import VoigtModel

# Create two Voigt models
voigt1 = VoigtModel(prefix='v1_')
voigt2 = VoigtModel(prefix='v2_')

# Composite model
model = voigt1 + voigt2

# Make parameters for the two Voigt peaks
params = model.make_params()

# Set initial guesses for the two peaks
params['v1_amplitude'].set(value=max(intensity), min=0)
params['v1_center'].set(value=binding_energy[np.argmax(intensity)], min=min(binding_energy), max=max(binding_energy))
params['v1_sigma'].set(value=1, min=0)
params['v1_gamma'].set(value=1, min=0)  # Voigt has an additional 'gamma' parameter for Lorentzian contribution

params['v2_amplitude'].set(value=0.5 * max(intensity), min=0)
params['v2_center'].set(value=binding_energy[np.argmax(intensity)] - 2, min=min(binding_energy), max=max(binding_energy))
params['v2_sigma'].set(value=1, min=0)
params['v2_gamma'].set(value=1, min=0)

# Perform the fit
result = model.fit(intensity, params, x=binding_energy)

# Plot the data and the fits
plt.plot(binding_energy, intensity, label='XPS Spectrum')
plt.plot(binding_energy, result.best_fit, label='Composite Voigt Fit', linestyle='--')

# Plot each individual component
plt.plot(binding_energy, result.eval_components()['v1_'], label='Voigt Peak 1', linestyle=':')
plt.plot(binding_energy, result.eval_components()['v2_'], label='Voigt Peak 2', linestyle=':')

plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

# Print the fitting report
print(result.fit_report())

### 4. Output & Interpretation
- The `fit_report()` gives you a summary of the fit, including parameter values, uncertainties, and goodness-of-fit metrics like the reduced chi-square.
- You can further fine-tune the model by adding constraints to parameters (e.g., fixing the position of a known peak).

This method is robust and allows detailed fitting, even for complex overlapping peaks.

## Working with `.xlsx` Files

### 1. Loading Data from Excel with Multiple Sheets
You can use `pandas.read_excel()` to load data from specific sheets or all sheets in an Excel file.

In [ ]:
import pandas as pd

# Load the Excel file, specifying the sheet name if needed
# To load a specific sheet
xps_data = pd.read_excel('xps_data.xlsx', sheet_name='Sheet1')

# If you want to load all sheets
all_sheets = pd.read_excel('xps_data.xlsx', sheet_name=None)

# Access a specific sheet from the dictionary of sheets
sheet1_data = all_sheets['Sheet1']

# Assuming 'Binding Energy' and 'Intensity' columns are present in the sheet
binding_energy = sheet1_data['Binding Energy']
intensity = sheet1_data['Intensity']

### 2. Working with Multiple Sheets
If you need to loop through all sheets and apply peak fitting to each one:

In [ ]:
for sheet_name, sheet_data in all_sheets.items():
    binding_energy = sheet_data['Binding Energy']
    intensity = sheet_data['Intensity']
    
    # Perform your peak fitting (code from previous steps) here
    print(f"Fitting for sheet: {sheet_name}")
    
    # Example of Gaussian fitting for each sheet
    result = gaussian_model.fit(intensity, params, x=binding_energy)
    
    # Plot the results for each sheet
    plt.plot(binding_energy, intensity, label=f'{sheet_name} XPS Spectrum')
    plt.plot(binding_energy, result.best_fit, label=f'{sheet_name} Gaussian Fit', linestyle='--')
    plt.xlabel('Binding Energy (eV)')
    plt.ylabel('Intensity (a.u.)')
    plt.legend()
    plt.show()

    # Print the fit report
    print(result.fit_report())

### 3. Combining Multiple Sheets into One Analysis
If you want to combine data from all sheets into one larger dataset (for instance, if the sheets represent different regions of the same material):

In [ ]:
combined_data = pd.concat(all_sheets.values())
binding_energy_combined = combined_data['Binding Energy']
intensity_combined = combined_data['Intensity']

# Perform your peak fitting on the combined dataset
result = gaussian_model.fit(intensity_combined, params, x=binding_energy_combined)

# Plot the combined result
plt.plot(binding_energy_combined, intensity_combined, label='Combined XPS Spectrum')
plt.plot(binding_energy_combined, result.best_fit, label='Combined Gaussian Fit', linestyle='--')
plt.xlabel('Binding Energy (eV)')
plt.ylabel('Intensity (a.u.)')
plt.legend()
plt.show()

# Print the combined fit report
print(result.fit_report())

### 4. Saving Results to a New Excel File
If you need to save the fit results for each sheet back into an Excel file:

In [ ]:
# Create a dictionary to store fit reports
fit_reports = {}

for sheet_name, sheet_data in all_sheets.items():
    binding_energy = sheet_data['Binding Energy']
    intensity = sheet_data['Intensity']
    
    # Perform the fit
    result = gaussian_model.fit(intensity, params, x=binding_energy)
    
    # Save the fit report for each sheet
    fit_reports[sheet_name] = result.fit_report()

# Convert the fit reports to a DataFrame and save them to a new Excel file
fit_reports_df = pd.DataFrame.from_dict(fit_reports, orient='index')
fit_reports_df.to_excel('fit_reports.xlsx')